In [ ]:
output_dir = "/content/drive/MyDrive/FYP/legal_argument_model_roberta_base"
    os.makedirs(output_dir, exist_ok=True)
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"\nModel and tokenizer saved to {output_dir}")

    # Function to make predictions
    def predict(text, model_path, tokenizer_path):
        # Load model and tokenizer
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

        # Tokenize input
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)

        # Get predictions
        with torch.no_grad():
            outputs = model(**inputs)

        # Get predicted class
        predicted_class_id = torch.argmax(outputs.logits, dim=1).item()
        return model.config.id2label[predicted_class_id]